In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

In [40]:
# smallest pval = 5.396032e-11   largest pval = .01
data = {'pval':  [1e-12, 1e-6,.01],
        'med': [0.4, 0.4,0.4],
        'gene': ['PTEN','PTEN','PTEN'],
        'cancer': ['Gbm','Hnscc','Brca']}
        

df = pd.DataFrame (data, columns = ['gene','pval','med','cancer'])

In [41]:
df

,gene,pval,med,cancer
0,PTEN,1.000000e-12,0.4,Gbm
1,PTEN1,1.000000e-06,0.4,Gbm
2,PTEN2,1.000000e-02,0.4,Gbm


In [25]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
gbm_df = pd.read_csv(root+R'\gbm_pval_medians.csv')


In [34]:
gbm_df = gbm_df.assign(cancer = 'Gbm')
gbm_df = gbm_df.iloc[0:20]
gbm_df

,Proteomics,Gbm_P_Value,Gbm_Median,cancer,size2,size
0,ARMH3_proteomics,5.396032e-11,-0.405134,Gbm,23.642772,70.928317
1,CUTC_proteomics,1.593480e-10,-0.553255,Gbm,22.559931,67.679793
2,PIP4K2A_proteomics,1.009419e-09,-0.838882,Gbm,20.713891,62.141673
3,CUL2_proteomics,1.122076e-09,-0.586396,Gbm,20.608085,61.824255
4,GDI2_proteomics,1.302273e-09,-0.610188,Gbm,20.459155,61.377465
5,ANXA7_proteomics,1.697911e-09,-0.490490,Gbm,20.193867,60.581601
6,DCTN1_proteomics,1.016640e-08,-0.382988,Gbm,18.404178,55.212533
7,PACS2_proteomics,1.123526e-08,-0.386279,Gbm,18.304209,54.912627
8,DOCK5_proteomics,2.754160e-08,-0.862819,Gbm,17.407568,52.222705
9,ACTR1A_proteomics,3.600370e-08,-0.491126,Gbm,17.139644,51.418932


In [33]:
p.plotCircleHeatMap(gbm_df, circle_var = 'Gbm_P_Value', color_var='Gbm_Median', x_axis= 'Proteomics', y_axis = 'cancer')
                    #plot_height=400, plot_width=1000)


Loading BokehJS ...

In [42]:
p.plotCircleHeatMap(df, circle_var = 'pval', color_var='med', x_axis= 'gene', y_axis = 'cancer')
                    #plot_height=400, plot_width=1000)


Loading BokehJS ...